In [83]:
import pandas as pd
from datetime import datetime

# autoreload modules when code is run
%load_ext autoreload
%autoreload 2

# local modules
import dataproject


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Data work

In [84]:
# 1.a load data without unnecessary data
raw_input = pd.read_csv('PL_all_seasons.csv', \
usecols=['Date','HomeTeam','AwayTeam','FTR','B365H','B365D','B365A'])

# 1.b convert date from string to datetimes
raw_input['Date'] = pd.to_datetime(raw_input['Date'], dayfirst = True)

# 1.c append seasons
# Add a new column named 'season' 
Type_new = pd.Series([])

# running a for loop and asigning some values to series 
for i in range(len(raw_input)): 
    if raw_input["Date"][i] < datetime(2010, 8, 10): 
        Type_new[i]="09/10"
  
    elif raw_input["Date"][i] < datetime(2011, 8, 10): 
        Type_new[i]="10/11"
  
    elif raw_input["Date"][i] < datetime(2012, 8, 10):  
        Type_new[i]="11/12"
  
    elif raw_input["Date"][i] < datetime(2013, 8, 10):  
        Type_new[i]="12/13"
  
    elif raw_input["Date"][i] < datetime(2014, 8, 10):  
        Type_new[i]="13/14"
    
    elif raw_input["Date"][i] < datetime(2015, 8, 10):  
        Type_new[i]="14/15"
  
    elif raw_input["Date"][i] < datetime(2016, 8, 10):  
        Type_new[i]="15/16"

    elif raw_input["Date"][i] < datetime(2017, 8, 10):  
        Type_new[i]="16/17"
  
    elif raw_input["Date"][i] < datetime(2018, 8, 10):  
        Type_new[i]="17/18"
  
    elif raw_input["Date"][i] < datetime(2019, 8, 10):  
        Type_new[i]="18/19"

    else: 
        season[i]= raw_input["Date"][i] 
  
        
        # inserting new column with values of list made above         
raw_input.insert(2, "Season", Type_new) 
# raw_input.head(10)
#####################################################################################################################

# 1.d Combine total data frame

df = pd.DataFrame(raw_input)

# 1.e add boolean of result Home, Draw, Away and then print df

df['H_true'] = df['FTR'].str.count("H")
df['D_true'] = df['FTR'].str.count("D")
df['A_true'] = df['FTR'].str.count("A")

# 1.f multiply bolean of Draw (D_true) with B365 odds of draw to get each games return for you

df['Return_H'] = ( df.loc[:, 'H_true'] * df.loc[:, 'B365H'] )
df['Return_D'] = ( df.loc[:, 'D_true'] * df.loc[:, 'B365D'] )
df['Return_A'] = ( df.loc[:, 'A_true'] * df.loc[:, 'B365A'] )

print(df)


Date     HomeTeam Season      AwayTeam FTR  B365H  B365D  B365A  \
0    2009-08-15  Aston Villa  09/10         Wigan   A   1.67   3.60   5.50   
1    2009-08-15    Blackburn  09/10      Man City   A   3.60   3.25   2.10   
2    2009-08-15       Bolton  09/10    Sunderland   A   2.25   3.25   3.25   
3    2009-08-15      Chelsea  09/10          Hull   H   1.17   6.50  21.00   
4    2009-08-15      Everton  09/10       Arsenal   A   3.20   3.25   2.30   
...         ...          ...    ...           ...  ..    ...    ...    ...   
3795 2019-05-12    Liverpool  18/19        Wolves   H   1.30   6.00  11.00   
3796 2019-05-12   Man United  18/19       Cardiff   A   1.28   6.50  11.00   
3797 2019-05-12  Southampton  18/19  Huddersfield   D   1.44   4.75   8.50   
3798 2019-05-12    Tottenham  18/19       Everton   D   2.20   3.50   3.50   
3799 2019-05-12      Watford  18/19      West Ham   A   2.25   3.75   3.20   

      H_true  D_true  A_true  Return_H  Return_D  Return_A  
0          0 

In [85]:
# Are there a 10 year returns on betting on either all away teams win (A), all games draw (D), or all home teams win (H)? Let's call these pure betting strategies H, D and A

In [86]:
# 2.a sum all individual returns to get you full season return for the full-season play A, D, and H

print(df.Return_A.sum())
print(df.Return_D.sum())
print(df.Return_H.sum())

#g. find the net return by subtracting the DKK 1 * 3800 games = DKK 3800 you have betted during the 10 season to find your gain/loss

print(df.Return_A.sum() - 3800)
print(df.Return_D.sum() - 3800)
print(df.Return_H.sum() - 3800)

#g. find the return in percentage by dividing you gain with the DKK 1 * 3800 games = DKK 3800 you have betted during the 10 season and subtract 1.

print(df.Return_A.sum() / 3800 - 1)
print(df.Return_D.sum() / 3800 - 1)
print(df.Return_H.sum() / 3800 - 1)


3504.25
3578.21
3804.42
-295.75
-221.78999999999996
4.420000000000073
-0.07782894736842105
-0.058365789473684204
0.001163157894736866


# How do the pure betting strategy returns look for each individual season?

In [87]:


# F. Group by seasons. Keeps booleans and results from H, D and A grouped by season. Why does, Date, AwayTeam, Hometeam disappear? Is it because they are strings?
Season_group = df.groupby(['Season','FTR']).sum()
drop_these = ['B365H','B365D','B365A']
Season_group.drop(drop_these, axis=1, inplace=True)

print(Season_group)

#####################################################################################################################

# G. Order the grouped dataframe. Sums the returns and boleans to make 6 rows into 2 rows (for presentation purposes).
Season_group['BetW'] = Season_group.H_true + Season_group.D_true + Season_group.A_true
Season_group['BetR'] = Season_group.Return_H + Season_group.Return_D + Season_group.Return_A

print(Season_group)

#####################################################################################################################

# H. Final dataframe. Here the net nominal return and the return in percent are calculated.
Bets = pd.DataFrame(Season_group, columns=['BetW', 'BetR'])
Bets['NetNomR'] = Bets.BetR - 380
Bets['Return_pct'] = (Bets.BetR / 380 -1)
Bets['Return_pct'] = pd.Series(["{0:.2f}%".format(val * 100) for val in Bets['Return_pct']],index = Bets.index)
Bets

H_true  D_true  A_true  Return_H  Return_D  Return_A
Season FTR                                                      
09/10  A         0       0      91      0.00      0.00    260.61
       D         0      96       0      0.00    351.49      0.00
       H       193       0       0    416.02      0.00      0.00
10/11  A         0       0      90      0.00      0.00    328.10
       D         0     111       0      0.00    406.39      0.00
       H       179       0       0    382.78      0.00      0.00
11/12  A         0       0     116      0.00      0.00    395.24
       D         0      93       0      0.00    348.96      0.00
       H       171       0       0    364.77      0.00      0.00
12/13  A         0       0     106      0.00      0.00    333.78
       D         0     108       0      0.00    397.04      0.00
       H       166       0       0    337.36      0.00      0.00
13/14  A         0       0     123      0.00      0.00    389.83
       D         0      78       0   

BetW    BetR  NetNomR Return_pct
Season FTR                                  
09/10  A      91  260.61  -119.39    -31.42%
       D      96  351.49   -28.51     -7.50%
       H     193  416.02    36.02      9.48%
10/11  A      90  328.10   -51.90    -13.66%
       D     111  406.39    26.39      6.94%
       H     179  382.78     2.78      0.73%
11/12  A     116  395.24    15.24      4.01%
       D      93  348.96   -31.04     -8.17%
       H     171  364.77   -15.23     -4.01%
12/13  A     106  333.78   -46.22    -12.16%
       D     108  397.04    17.04      4.48%
       H     166  337.36   -42.64    -11.22%
13/14  A     123  389.83     9.83      2.59%
       D      78  304.97   -75.03    -19.74%
       H     179  388.39     8.39      2.21%
14/15  A     119  402.90    22.90      6.03%
       D      96  364.73   -15.27     -4.02%
       H     174  377.28    -2.72     -0.72%
15/16  A     112  390.75    10.75      2.83%
       D     104  389.95     9.95      2.62%
       H     155  342.73   -37.27     -9.81%
16/17  A     109  307.91   -72.09    -18.97%
       D      84  334.33   -45.67    -12.02%
       H     187  404.93    24.93      6.56%
17/18  A     108  322.58   -57.42    -15.11%
       D      99  398.84    18.84      4.96%
       H     173  390.98    10.98      2.89%
18/19  A     128  372.55    -7.45     -1.96%
       D      71  281.51   -98.49    -25.92%
       H     181  399.18    19.18      5.05%